#Function calling 

## Classifying the question asked into a class of questions
- once classfied the appropriate set of tools for that class can be exposed to LLM 
- This helps tio reducxe the total number of tools exposed to LLM and reduce context data

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm) 
- cohere.command-a-03-2025
- cohere.command-r-08-2024
- cohere.command-r-plus-08-2024


Questions use #generative-ai-users  or ##igiu-innovation-lab slack channels 

 if you have errors running sample code reach out for help in #igiu-ai-learning
 


### Set up variables

In [3]:
# set up the  variables


from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import *
import oci
import json, os 
from dotenv import load_dotenv
from envyaml import EnvYAML

#####
#make sure your sandbox.yaml file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

SANDBOX_CONFIG_FILE = "sandbox.yaml"
load_dotenv()

LLM_MODEL = "cohere.command-r-plus-08-2024" 
PREAMBLE = """
        answer in three bullets. respond in hindi 
"""
MESSAGE = """
        "why is indian cricket team so good"
"""

llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

llm_client = None
llm_payload = None

### load the profile

In [4]:

scfg = EnvYAML(SANDBOX_CONFIG_FILE)
if scfg is None or "oci" not in scfg or "bucket" not in scfg:
    raise RuntimeError("Invalid sandbox configuration.")

#read the oci config
config = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])

### set up Chat Request

In [5]:


cohere_chat_request = CohereChatRequest()
cohere_chat_request.preamble_override = """
        
        You are a call classifier you carefully analyze teh question and classify it into one of the following categories. 
        you then return just the category in response
        
        categories: billing, outage, program, service
        
        eg: 
        question:  can i pay my bill by creditcard
        answer: billing
        question: i need to cancel my service as i am moving
        answer: service
        question: when will by power come back on?
        answer: outage
        
        """
cohere_chat_request.is_stream = False 
cohere_chat_request.max_tokens = 500
cohere_chat_request.temperature = 0.75
cohere_chat_request.top_p = 0.7
cohere_chat_request.frequency_penalty = 1.0


## Set up chat details

In [9]:

chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = scfg["oci"]["compartment"]
chat_detail.chat_request = cohere_chat_request

### get the LLM client 

In [10]:

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

### Ask the question 

In [11]:
cohere_chat_request.message = "why is my bill so high"

llm_response = llm_client.chat(chat_detail)
llm_text = llm_response.data.chat_response.text

### print response 

In [13]:
print("**************************Chat Result**************************")
#llm_text = llm_response.data.chat_response.text
print(llm_response.data.chat_response.text)

**************************Chat Result**************************
billing


## Exercise 


Merge your RAG Agent & Single step tool into same code and use clasiification to decide which to invoke
eg: 
1. If the queston is about weather, cll the single step tool
2. if the call is about batteries use the RAG to answer
3. if its about something else say you can only answer questions about weather or batteries

